In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

In [2]:
link1 = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"
link2 = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"
def get_wikidata(link):

    source = urllib.request.urlopen(link).read()
    soup = bs.BeautifulSoup(source,'lxml')
    tables = soup.find_all('table',class_='wikitable sortable')

    df1 = pd.read_html(str(tables[0]))[0]
    df2 = pd.read_html(str(tables[1]))[0]
    df3 = pd.read_html(str(tables[2]))[0]
    df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0]

    df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

    return df

df1 = get_wikidata(link1)
df2 = get_wikidata(link2)

In [3]:
df2

,Opening,Opening.1,Title,Production company,Cast and crew,Ref,Ref.
0,JANUARY,1,Shadow in the Cloud,Vertical Entertainment,Roseanne Liang (director/screenplay); Max Land...,[2],NaN
1,JANUARY,13,The White Tiger,Netflix,Ramin Bahrani (director/screenplay); Adarsh Go...,NaN,NaN
2,JANUARY,14,Locked Down,HBO Max / Warner Bros. Pictures,Doug Liman (director); Steven Knight (screenpl...,[3],NaN
3,JANUARY,15,The Dig,Netflix / Clerkenwell Films,Simon Stone (director); Moira Buffini (screenp...,[4],NaN
4,JANUARY,15,Outside the Wire,Netflix,"Mikael Håfström (director); Rob Yescombe, Rowa...",[5],NaN
...,...,...,...,...,...,...,...
238,DECEMBER,22,The Matrix 4,HBO Max / Warner Bros. Pictures / Village Road...,Lana Wachowski (director/screenplay); Aleksand...,NaN,[171]
239,DECEMBER,22,Sing 2,Universal Pictures / Illumination,Garth Jennings (director/screenplay); Matthew ...,NaN,[172]
240,DECEMBER,22,The King's Man,20th Century Studios / Marv Films,Matthew Vaughn (director/screenplay); Karl Gaj...,NaN,[110]
241,DECEMBER,22,Downton Abbey 2,Focus Features / Carnival Films,Simon Curtis (director); Julian Fellowes (scre...,NaN,[173]


In [8]:
from tmdbv3api import Movie
from tmdbv3api import TMDb
tmdb_movie = Movie() 
tmdb = TMDb()
tmdb.api_key = '7767da907c5a44d336bd201316cdb8ab'
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

def ordering_data(df):
    dfs = df[['Title','Cast and crew']]


    dfs['genres'] = dfs['Title'].map(lambda x: get_genre(str(x))) 
    dfs['director_name'] = dfs['Cast and crew'].map(lambda x: get_director(str(x)))
    dfs['actor_1_name'] = dfs['Cast and crew'].map(lambda x: get_actor1(str(x)))
    dfs['actor_2_name'] = dfs['Cast and crew'].map(lambda x: get_actor2(str(x)))
    dfs['actor_3_name'] = dfs['Cast and crew'].map(lambda x: get_actor3(str(x)))
    dfs = dfs.rename(columns={'Title':'movie_title'})

    new_df = dfs.loc[:,['movie_title','genres','director_name','actor_1_name','actor_2_name','actor_3_name',]]
    new_df['comb'] = new_df['actor_1_name'] + ' ' + new_df['actor_2_name'] + ' '+ new_df['actor_3_name'] + ' '+ new_df['director_name'] +' ' + new_df['genres']
    new_df = new_df.dropna(how='any')
    new_df['movie_title'] = new_df['movie_title'].str.lower()

    return new_df

In [9]:
dfs_1 = ordering_data(df1)
dfs_2 = ordering_data(df2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [10]:
dfs_2

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,comb
0,shadow in the cloud,Horror Action War,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Chloë Grace Moretz Taylor John Smith Beulah Ko...
1,the white tiger,Crime Drama,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Adarsh Gourav Rajkummar Rao Priyanka Chopra Jo...
2,locked down,Comedy Crime Drama,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Anne Hathaway Chiwetel Ejiofor Stephen Merchan...
3,the dig,Drama History,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James,Carey Mulligan Ralph Fiennes Lily James Simon ...
4,outside the wire,Thriller Action Science Fiction,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham,Anthony Mackie Damson Idris Emily Beecham Mika...
...,...,...,...,...,...,...,...
238,the matrix 4,Action Science Fiction,Lana Wachowski,Keanu Reeves,Carrie-Anne Moss,Yahya Abdul-Mateen II,Keanu Reeves Carrie-Anne Moss Yahya Abdul-Mate...
239,sing 2,Music Animation Comedy Family Drama,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson,Matthew McConaughey Reese Witherspoon Scarlett...
240,the king's man,Action Adventure Comedy,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
241,downton abbey 2,Drama,Simon Curtis,Hugh Bonneville,Jim Carter,Michelle Dockery,Hugh Bonneville Jim Carter Michelle Dockery Si...


In [11]:
dfs_1

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,comb
0,the grudge,Horror Mystery Thriller,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Andrea Riseborough Demián Bichir John Cho Nico...
1,underwater,Action Horror Science Fiction Thriller,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Kristen Stewart Vincent Cassel Jessica Henwick...
2,like a boss,Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,three christs,Drama,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Richard Gere Peter Dinklage Walton Goggins Jon...
4,inherit the viper,Drama Thriller Crime,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Josh Hartnett Margarita Levieva Chandler Riggs...
...,...,...,...,...,...,...,...
266,soul,Family Animation Drama Music Fantasy,Pete Docter,Jamie Foxx,Tina Fey,Graham Norton,Jamie Foxx Tina Fey Graham Norton Pete Docter ...
267,we can be heroes,Action Fantasy Family Comedy,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...
269,one night in miami...,Drama,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...
270,promising young woman,Thriller Crime Drama,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Carey Mulligan Bo Burnham Alison Brie Emerald ...


In [12]:
old_df = pd.read_csv('final_data.csv')
old_df

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,comb
0,avatar,Action Adventure Fantasy Sci-Fi,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,CCH Pounder Joel David Moore Wes Studi James C...
1,pirates of the caribbean: at world's end,Action Adventure Fantasy,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,spectre,Action Adventure Thriller,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,the dark knight rises,Action Thriller,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,star wars: episode vii - the force awakens ...,Documentary,Doug Walker,Doug Walker,Rob Walker,unknown,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5867,spies in disguise,Animation Action Adventure Comedy Family,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones,Will Smith Tom Holland Rashida Jones Nick Brun...
5868,little women,Drama Romance,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Saoirse Ronan Emma Watson Florence Pugh Greta ...
5869,1917,War Drama Action Thriller,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,George MacKay Dean-Charles Chapman Mark Strong...
5870,just mercy,Drama Crime History,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Michael B. Jordan Jamie Foxx Brie Larson Desti...


In [15]:
df_12 = dfs_1.append(dfs_2,ignore_index=True)
df_final = old_df.append(df_12,ignore_index=True)


In [16]:
df_final

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,comb
0,avatar,Action Adventure Fantasy Sci-Fi,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,CCH Pounder Joel David Moore Wes Studi James C...
1,pirates of the caribbean: at world's end,Action Adventure Fantasy,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,spectre,Action Adventure Thriller,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,the dark knight rises,Action Thriller,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,star wars: episode vii - the force awakens ...,Documentary,Doug Walker,Doug Walker,Rob Walker,unknown,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6340,the matrix 4,Action Science Fiction,Lana Wachowski,Keanu Reeves,Carrie-Anne Moss,Yahya Abdul-Mateen II,Keanu Reeves Carrie-Anne Moss Yahya Abdul-Mate...
6341,sing 2,Music Animation Comedy Family Drama,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson,Matthew McConaughey Reese Witherspoon Scarlett...
6342,the king's man,Action Adventure Comedy,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
6343,downton abbey 2,Drama,Simon Curtis,Hugh Bonneville,Jim Carter,Michelle Dockery,Hugh Bonneville Jim Carter Michelle Dockery Si...


In [17]:
df_final.to_csv('main_data.csv',index=False)